# Hello Loopy: Computing a Rank-One Matrix

## Setup Code

In [16]:
import numpy as np
import pyopencl as cl
import pyopencl.array
import pyopencl.clrandom
import loopy as lp

In [21]:
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

In [22]:
n = 1024
a = cl.clrandom.rand(queue, n, dtype=np.float32)
b = cl.clrandom.rand(queue, n, dtype=np.float32)

## The Initial Kernel

In [23]:
knl = lp.make_kernel(
    "{[i,j]: 0<=i,j<n}",
    "c[i, j] = a[i]*b[j]")

In [24]:
knl = lp.set_options(knl, write_cl=True)
evt, (mat,) = knl(queue, a=a, b=b)

#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))

__kernel void __attribute__ ((reqd_work_group_size(1, 1, 1))) loopy_kernel(__global float const *__restrict__ a, __global float const *__restrict__ b, __global float *__restrict__ c, int const n)
{
  for (int j = 0; j <= -1 + n; ++j)
    for (int i = 0; i <= -1 + n; ++i)
      c[n * i + j] = a[i] * b[j];
}



## Transforming kernels: Implementation Tags

Every loop axis ("iname") comes with an *implementation tag*.

In [29]:
isplit_knl = knl
#isplit_knl = lp.assume(isplit_knl, "n mod 4 = 0")
isplit_knl = lp.split_iname(isplit_knl, "i", 4)
#isplit_knl = lp.tag_inames(isplit_knl, {"i_inner": "unr"})

evt, (mat,) = isplit_knl(queue, a=a, b=b)

#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))
#define int_floor_div_pos_b(a,b) (                 ( (a) - ( ((a)<0) ? ((b)-1) : 0 )  ) / (b)                 )

__kernel void __attribute__ ((reqd_work_group_size(1, 1, 1))) loopy_kernel(__global float const *__restrict__ a, __global float const *__restrict__ b, __global float *__restrict__ c, int const n)
{
  for (int j = 0; j <= -1 + n; ++j)
    for (int i_outer = 0; i_outer <= -1 + int_floor_div_pos_b(3 + n, 4); ++i_outer)
      for (int i_inner = 0; i_inner <= (-4 + n + -4 * i_outer >= 0 ? 3 : -1 + n + -4 * i_outer); ++i_inner)
        c[n * (4 * i_outer + i_inner) + j] = a[4 * i_outer + i_inner] * b[j];
}



----
"Map to GPU hw axis" is an iname tag as well.

Use shortcuts for less typing:

In [8]:
split_knl = knl
split_knl = lp.split_iname(split_knl, "i", 16,
        outer_tag="g.0", inner_tag="l.0")
split_knl = lp.split_iname(split_knl, "j", 16,
        outer_tag="g.1", inner_tag="l.1")

evt, (mat,) = split_knl(queue, a=a, b=b)

#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))

__kernel void __attribute__ ((reqd_work_group_size(16, 16, 1))) loopy_kernel(__global float const *__restrict__ a, __global float const *__restrict__ b, __global float *__restrict__ c, int const n)
{
  if (-1 + -16 * gid(1) + -1 * lid(1) + n >= 0 && -1 + -16 * gid(0) + -1 * lid(0) + n >= 0)
    c[n * (16 * gid(0) + lid(0)) + 16 * gid(1) + lid(1)] = a[16 * gid(0) + lid(0)] * b[16 * gid(1) + lid(1)];
}



## Targeting CPUs/KNL

[ISPC](https://ispc.github.io/)'s "low-level" interface provides access to SSE, AVX2, AVX512 (including Knight's hardware) from a scalar CUDA program model:

In [19]:
ispc_knl = knl

ispc_knl = ispc_knl.copy(target=lp.ISPCTarget())

ispc_knl = lp.split_iname(ispc_knl, "j", 16,
        outer_tag="g.0", inner_tag="l.0")
ispc_knl = lp.add_and_infer_dtypes(ispc_knl, dict(a=np.float32, b=np.float32))

print(lp.generate_code_v2(ispc_knl).all_code())

#define int_floor_div_pos_b(a,b) (                 ( (a) - ( ((a)<0) ? ((b)-1) : 0 )  ) / (b)                 )

task void loopy_kernel_inner(uniform float const *uniform a, uniform float const *uniform b, uniform float *uniform c, uniform int32 const n)
{
  if (-1 + -16 * ((uniform int32) taskIndex0) + -1 * (varying int32) programIndex + n >= 0)
    for (uniform int32 i = 0; i <= -1 + n; ++i)
      c[n * i + 16 * ((uniform int32) taskIndex0) + (varying int32) programIndex] = a[i] * b[16 * ((uniform int32) taskIndex0) + (varying int32) programIndex];
}

export void loopy_kernel(uniform float const *uniform a, uniform float const *uniform b, uniform float *uniform c, uniform int32 const n)
{
  assert(programCount == (16));
  launch[int_floor_div_pos_b(15 + n, 16)] loopy_kernel_inner(a, b, c, n);

}


## Transforming kernels: Leveraging data reuse

Would like to fetch entire "access footprint" of a loop.

In [22]:
fetch_knl = split_knl

fetch_knl = lp.add_prefetch(fetch_knl, "a", ["i_inner"])
fetch_knl = lp.add_prefetch(fetch_knl, "b", ["j_inner"])

evt, (mat,) = fetch_knl(queue, a=a, b=b)

#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))

__kernel void __attribute__ ((reqd_work_group_size(16, 16, 1))) loopy_kernel(__global float const *__restrict__ a, __global float const *__restrict__ b, __global float *__restrict__ c, int const n)
{
  __local float a_fetch[16];
  __local float b_fetch[16];

  if (-1 + -16 * gid(1) + -1 * lid(0) + n >= 0)
    b_fetch[lid(0)] = b[16 * gid(1) + lid(0)];
  if (-1 + -16 * gid(0) + -1 * lid(0) + n >= 0)
    a_fetch[lid(0)] = a[16 * gid(0) + lid(0)];
  barrier(CLK_LOCAL_MEM_FENCE) /* for a_fetch (insn depends on a_fetch_rule) */;
  if (-1 + -16 * gid(1) + -1 * lid(1) + n >= 0 && -1 + -16 * gid(0) + -1 * lid(0) + n >= 0)
    c[n * (16 * gid(0) + lid(0)) + 16 * gid(1) + lid(1)] = a_fetch[lid(0)] * b_fetch[lid(1)];
}

